In [ ]:
import xarray as xr
import numpy as np

# Last Deglaciation

In [ ]:
ds = xr.open_dataset("../runs/glac1d/outputs_gridded/0/laea_5km_eurasia.nc")

shape = ds.ice_thickness.values.shape
x = ds.x.values
y = ds.y.values
times = ds.time.values
crs = ds.rio.crs

dA = np.abs(x[1] - x[0]) * np.abs(y[1] - y[0])

In [ ]:
def package_output(run_dir, n_members, times, x, y, crs, attrs=None):
    run_dir = Path(run_dir)
    
    ice_thickness = np.zeros((n_members,*shape))
    shearstress = np.zeros((n_members,*shape))
    topography = np.zeros((shape))

    for t_idx, time in enumerate(times):
        topo_ds = xr.open_dataset(run_dir / f"inputs/topography/{time}/topography.nc")
        topography[t_idx] = topo_ds.z.values

    for idx in tqdm(range(n_members)):
        ice_ds = xr.open_dataset(run_dir / f"outputs_gridded/{idx}/laea_5km_eurasia.nc")
        ice_thickness[idx] = ice_ds.ice_thickness.values
        for t_idx, time in enumerate(times):
            shearstress_ds = xr.open_dataset(run_dir / f"inputs/shearstress/{time}/{idx}/shearstress.nc")
            shearstress[idx, t_idx] = shearstress_ds.z.values


    ds = xr.Dataset(
        data_vars=dict(
            ice_thickness=(
                ["ensemble_index", "time", "y", "x"],
                ice_thickness,
            ),
            bedrock_topography=(
                ["time", "y", "x"],
                topography,
            ),
            basal_shearstress=(
                ["ensemble_index", "time", "y", "x"],
                shearstress,
            ),
        ),
        coords={
            "ensemble_index": list(range(n_members)),
            "time": times,
            "y": y,
            "x": x,
        },
        attrs=attrs,
    )
    ds.rio.write_crs(crs, inplace=True)
    return ds



In [ ]:
glad1d_ds = package_output(
    run_dir = "paper1_data/icesheet_runs/glac1d/",
    n_members = 200,
    times = times,
    x = x,
    y = y,
    crs = crs
)
glad1d_ds.to_netcdf(f"glac1d_icesheet.nc")

In [ ]:
glad1d_corners_ds = package_output(
    run_dir = "paper1_data/icesheet_runs/glac1d/corners/",
    n_members = 128,
    times = times,
    x = x,
    y = y,
    crs = crs
)
glad1d_corners_ds.to_netcdf(f"glac1d_icesheet_corners.nc")

In [ ]:
ice6g_ds = package_output(
    run_dir = "paper1_data/icesheet_runs/ice6g/",
    n_members = 200,
    times = times,
    x = x,
    y = y,
    crs = crs
)
ice6g_ds.to_netcdf(f"glac1d_icesheet.nc")

In [ ]:
ice6g_corners_ds = package_output(
    run_dir = "paper1_data/icesheet_runs/ice6g/corners",
    n_members = 128,
    times = times,
    x = x,
    y = y,
    crs = crs
)
ice6g_corners_ds.to_netcdf(f"glac1d_icesheet.nc")

# PGM

In [ ]:
ds = xr.open_dataset("paper1_data/icesheet_runs/batchelor/outputs_gridded/0/laea_5km_eurasia.nc")

shape = ds.ice_thickness.values.shape
x = ds.x.values
y = ds.y.values
time = ds.time.values

dA = np.abs(x[1] - x[0]) * np.abs(y[1] - y[0])

In [ ]:
ice_thickness = np.zeros((200,*shape))
shearstress = np.zeros((200,*shape))
topography = np.zeros((shape))

topo_ds = xr.open_dataset(f"paper1_data/icesheet_runs/batchelor/topo_iteration_0/inputs/topography/topography.nc")
topography[0] = topo_ds.z.values

for idx in tqdm(range(200)):
    ice_ds = xr.open_dataset(f"paper1_data/icesheet_runs/batchelor/topo_iteration_0/outputs_gridded/{idx}/laea_5km_eurasia.nc")
    ice_thickness[idx] = ice_ds.ice_thickness.values
    
    shearstress_ds = xr.open_dataset(f"paper1_data/icesheet_runs/batchelor/topo_iteration_0/inputs/shearstress/-140000/{idx}/shearstress.nc")
    shearstress[idx] = shearstress_ds.z.values

batchelor_topo_0_ds = xr.Dataset(
    data_vars=dict(
        ice_thickness=(
            ["ensemble_index", "time", "y", "x"],
            ice_thickness,
        ),
        bedrock_topography=(
            ["time", "y", "x"],
            topography,
        ),
        basal_shearstress=(
            ["ensemble_index", "time", "y", "x"],
            shearstress,
        ),
    ),
    coords={
        "ensemble_index": list(range(200)),
        "time": time,
        "y": y,
        "x": x,
    },
    attrs=None,
)
batchelor_topo_0_ds.rio.write_crs(ds.rio.crs, inplace=True)
batchelor_topo_0_ds.to_netcdf(f"batchelor_topo_0.nc")

In [ ]:
ice_thickness = np.zeros((200,*shape))
shearstress = np.zeros((200,*shape))
topography = np.zeros((200,*shape))

for idx in tqdm(range(200)):
    ice_ds = xr.open_dataset(f"paper1_data/icesheet_runs/batchelor/topo_iteration_1/outputs_gridded/{idx}/laea_5km_eurasia.nc")
    ice_thickness[idx] = ice_ds.ice_thickness.values
    
    shearstress_ds = xr.open_dataset(f"paper1_data/icesheet_runs/batchelor/topo_iteration_1/inputs/shearstress/{idx}/shearstress.nc")
    shearstress[idx] = shearstress_ds.z.values
    
    topo_ds = xr.open_dataset(f"paper1_data/icesheet_runs/batchelor/topo_iteration_1/inputs/topography/-140000/{idx}/topography.nc")
    topography[idx] = topo_ds.z.values

batchelor_topo_1_ds = xr.Dataset(
    data_vars=dict(
        ice_thickness=(
            ["ensemble_index", "time", "y", "x"],
            ice_thickness,
        ),
        bedrock_topography=(
            ["ensemble_index", "time", "y", "x"],
            topography,
        ),
        basal_shearstress=(
            ["ensemble_index", "time", "y", "x"],
            shearstress,
        ),
    ),
    coords={
        "ensemble_index": list(range(200)),
        "time": time,
        "y": y,
        "x": x,
    },
    attrs=None,
)
batchelor_topo_1_ds.rio.write_crs(ds.rio.crs, inplace=True)
batchelor_topo_1_ds.to_netcdf(f"batchelor_topo_1.nc")